#**Projet - Régression logistique**

But : implémenter la régression logistique et mettre en oeuvre une méthodologie solide

Cours : M1 MIAGE apprentissage (Dauphine) - 2023-2024

Projet en binôme

A rendre :
*   Code ayant permis de résoudre les différentes questions (Notebook Python) - commenté pour permettre son exécution
*   Rapport en PDF (moins de 20 pages)

Date : le 01/12/2023 (avant 20h - heure de Paris)

In [17]:
# libairies utiles pour ce cours

import sklearn as sk
from sklearn import datasets, model_selection
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.preprocessing import StandardScaler

import copy
import pandas as pd
from scipy import stats

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

## I Regression logistique (8pts)

1.   En vous appuyant sur le code écrit pour le TP 2 (notamment la descente de gradient), implémenter la régression logistique (2pts)

  /!\ /!\  *il faudra modifier le code ou le problème d'optimisation car dans le TP2, on minimisait une fonction et là, on maximise une fonction* /!\ /!\
2. Calculer $\beta$, les paramètres de la frontière de décision (1 pt)
8. prédire la classe du point $x = \left( \begin{array}{c}
0 \\
0 \end{array} \right)$ (1 pt)
9. Représenter la frontière de décision (en ré-utilisant le code écrit pour le TP 1) (1pt)
10. Développer une fonction de calcul de taux d'erreur d'un modèle. Appliquer ce code pour estimer le taux d'erreur (en apprentissage et en test) de votre modèle de régression logistique. (1pt)
11. Comparer ces résultats (frontière de décisions et taux d'erreurs) à ceux obtenus sur ces données par votre code de LDA (développé pour le TP 1) - commenter (2 pts)

## II Méthodologie (5pts)

7. inclure une ligne permettant d'ajouter un outlier dans les données d'apprentissage (ligne commentée dans le bloc de génération de données).
Comparez les résultats de vos codes de régression logistique et de LDA sur ces données. Commenter (1pt)
8. en plus des taux d'erreurs en apprentissage et en test, appliquer une statégie de validation croisée à n blocs et reporter les taux d'erreur (moyenne et variance) des deux modèles. (2pts)
9. modifier le code de génération de données (en augmentant le nombre de dimensions et en ajoutant des variables correlées). Comparer le comportement des deux modèles (2pts)      *(on commentera à nouveau la ligne de génération d'outlier pour cette question)*



## III A vous de jouer (7 pts)
10. Trouver/Générer/Récolter des données adéquates pour l'application d'un algorithme de classification binaire. Décrire ces données (origine, nombre et types de variables, analyse statistique des variables, nombre d'observations, contexte,...) (2pts)
11. Appliquer vos codes de régression logistique et de LDA et reporter les performances obtenues (en apprentissage, en test ou en validation croisées) et commenter (1pts)
12. Trouver une autre méthode à laquelle comparer les deux codes que vous avez développés, décrire (brièvement) le fonctionnement de cette troisème approche et l'appliquer sur les données (de la question 10). On pourra développer soit-même le code ou utiliser une librairie existante. Comparer et discuter les performances obtenues (4pts)

In [18]:
def statistic_data(X,y):
  print("Number of observations: ",X.shape[0])
  print("Number of features: ",X.shape[1])
  stat_0, p_value_0 = stats.normaltest(X[y==0]) #Tester si la distribution de X des observations de classe 0 est approximativement normale
  stat_1, p_value_1 = stats.normaltest(X[y==1]) #Tester si la distribution de X des observations de classe 0 est approximativement normale
  print("Is distribution of X approximately normal in class 0: ",p_value_0>= 0.05)
  print("Is distribution of X approximately normal in class 1: ",p_value_1>= 0.05)

def calculate_y_hat_logistic_regression(beta, Xb):
  return 1/(1+np.exp(-np.dot(Xb,beta)))

def calculate_y_hat_using_LDA (w,b,X):
  return X@w+b

# I.1
def gradient_ascent(Xb,y):
  itermax=10000 # nombre d'itérations maximum
  i = 1
  notconv=True #critere de convergence

  eta=1e-3 #pas de la ascension de gradient

  beta=np.random.normal(size=(Xb.shape[1],1)) #initialisation aléatoire de beta

  losses=[] #calcul de la fonction cout à chaque iteration
  loss_old = np.inf
  y = y.reshape(-1,1)
  reason="attain itermax"
  while (i <= itermax) and (notconv):
    beta_old=beta

    y_hat=calculate_y_hat_logistic_regression(beta, Xb)
    grad=Xb.T@(y-y_hat)
    beta=beta_old + eta * grad

    loss=-np.mean((y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)))
    losses.append(loss)

    if np.linalg.norm(beta_old-beta)<1e-10: # critère d'arrêt 1
      notconv=False
      reason="diff iterate"

    if loss>loss_old: # critère d'arrêt 2
      notconv=False
      reason = "falling loss"

    i += 1
    loss_old=loss
  return beta,losses, reason

def LDA(X_train,y_train):
  mu_0 = np.mean(X_train[y_train==0,:],axis=0)
  mu_1 = np.mean(X_train[y_train==1,:],axis=0)

  pi_0 = np.mean(y_train==0)
  pi_1 = np.mean(y_train==1)

  Xtemp = np.concatenate((X_train[y_train==0,:]-mu_0,X_train[y_train==1,:]-mu_1),axis=0)
  Sigma = (Xtemp.T)@Xtemp/X_train.shape[0]
  iSigma= np.linalg.inv(Sigma)
  w = iSigma@(mu_0-mu_1)
  b = -1/2 * (mu_0-mu_1).T@iSigma@(mu_0+mu_1) + np.log(pi_0/pi_1)
  return w,b

def draw_decision_boundary_config(X,y):
  x1_min, x1_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
  x2_min, x2_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
  h = .02

  x1_axis = np.arange(x1_min, x1_max, h)
  x2_axis = np.arange(x2_min, x2_max, h)

  x1_coors, x2_coors = np.meshgrid(x1_axis, x2_axis)

  return x1_axis,x2_axis, x1_coors, x2_coors

def draw_decision_boundary_logistic_regression(beta,X,y,xlabel,ylabel):
  x1_axis,x2_axis, x1_coors, x2_coors=draw_decision_boundary_config(X,y)
  big_x = np.column_stack((x1_coors.ravel(), x2_coors.ravel(), np.ones(x1_coors.ravel().shape[0])))

  figure = plt.figure()
  decision_boundary_coor = calculate_y_hat_logistic_regression(beta, big_x).reshape(x1_coors.shape)

  cm_bright = ListedColormap(['#FF0000', '#0000FF'])
  plt.pcolormesh(x1_axis, x2_axis, decision_boundary_coor>=0.5, cmap=cm_bright, shading='auto', alpha=0.6)
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
  plt.title("Frontière de décision Régression Logistique")
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

def draw_decision_boundary_LDA(w,b,X,y,xlabel,ylabel):
  x1_axis,x2_axis, x1_coors, x2_coors=draw_decision_boundary_config(X,y)
  big_x = np.column_stack((x1_coors.ravel(), x2_coors.ravel()))

  figure = plt.figure()
  decision_boundary_coor =calculate_y_hat_using_LDA(w,b,big_x).reshape(x1_coors.shape)

  cm_bright = ListedColormap(['#FF0000', '#0000FF'])
  plt.pcolormesh(x1_axis, x2_axis, decision_boundary_coor<0, cmap=cm_bright, shading='auto', alpha=0.6)
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
  plt.title("Frontière de décision LDA")
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

def draw_decision_boundary_decision_tree(clf,X,y,xlabel,ylabel):
  x1_axis,x2_axis, x1_coors, x2_coors=draw_decision_boundary_config(X,y)
  big_x = np.column_stack((x1_coors.ravel(), x2_coors.ravel()))

  figure = plt.figure()
  decision_boundary_coor =clf.predict(big_x).reshape(x1_coors.shape)

  cm_bright = ListedColormap(['#FF0000', '#0000FF'])
  plt.pcolormesh(x1_axis, x2_axis, decision_boundary_coor==1, cmap=cm_bright, shading='auto', alpha=0.6)
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
  plt.title("Frontière de décision Decision Tree")
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

def calculate_logistic_regression_error_ratio(beta,Xb,y):
  y_hat=calculate_y_hat_logistic_regression(beta, Xb)
  y_hat[y_hat>=0.5]=1
  y_hat[y_hat<0.5]=0
  nb_correct_prediction=0
  for i in range (0,len(y)):
    if(y_hat[i]==y[i]):
      nb_correct_prediction+=1
  return (len(y)-nb_correct_prediction)/len(y)

def calculate_LDA_error_ratio(w,b,X,y):
  y_hat=calculate_y_hat_using_LDA(w,b,X)
  y_hat[y_hat>=0]=0
  y_hat[y_hat<0]=1
  nb_correct_prediction=0
  for i in range (0,len(y)):
    if(y_hat[i]==y[i]):
      nb_correct_prediction+=1
  return (len(y)-nb_correct_prediction)/len(y)


def calculate_decision_tree_error_ratio(clf,X,y):
  y_hat=clf.predict(X)
  nb_correct_prediction=0
  for i in range (0,len(y)):
    if(y_hat[i]==y[i]):
      nb_correct_prediction+=1
  return (len(y)-nb_correct_prediction)/len(y)

def predict_class_of_point_using_logistic_regression(beta,xb):
  y_hat = calculate_y_hat_logistic_regression(beta, xb)
  if(y_hat>=0.5):
    print("Logistic regression says that x belongs to class 1")
  else:
    print("Logistic regression says that x belongs to class 0")

def predict_class_of_point_using_LDA(w,b,xb):
  y_hat = calculate_y_hat_using_LDA(w,b,xb)
  if(y_hat>=0):
    print("LDA says that x belongs to class 0")
  else:
    print("LDA says that x belongs to class 1")

def report_error_ratio(X,y):
  loo = model_selection.LeaveOneOut()
  errors_logistic_regression=[]
  errors_LDA=[]
  errors_decision_tree=[]

  for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    Xb=np.column_stack((X_train,np.ones(X_train.shape[0])))
    Xtb=np.column_stack((X_test,np.ones(X_test.shape[0])))

    clf = DecisionTreeClassifier(random_state=0)
    clf.fit(X_train, y_train)
    errors_decision_tree.append(calculate_decision_tree_error_ratio(clf,X_test,y_test))

    beta,_,_=gradient_ascent(Xb,y_train)
    errors_logistic_regression.append(calculate_logistic_regression_error_ratio(beta,Xtb,y_test))
    #Parfois, on se trouve dans un cas où matrice Sigma n'est pas inversible. Si c'est le cas, j'exclus le résultat de cette itération du tableau d'erreurs de LDA
    #Ce problème est peut-être causé par:
    #- Multicolinéarité : certaines variables sont fortement corrélées entre elles, la matrice de covariance peut devenir singulière, ce qui signifie qu'elle n'est pas inversible.
    #- Données redondantes : Si les variables sont redondantes, c'est-à-dire qu'elles ne fournissent pas d'informations supplémentaires, cela peut conduire à une matrice de covariance singulière.
    try:
      w,b=LDA(X_train,y_train)
      errors_LDA.append(calculate_LDA_error_ratio(w,b,X_test,y_test))
    except:
      continue

  mean_error_logistic_regression=np.mean(errors_logistic_regression)
  var_error_logistic_regression = np.var(errors_logistic_regression)

  mean_error_LDA=np.mean(errors_LDA)
  var_error_LDA = np.var(errors_LDA)

  mean_error_decision_tree= np.mean(errors_decision_tree)
  var_error_decision_tree = np.var(errors_decision_tree)

  return mean_error_logistic_regression,var_error_logistic_regression,mean_error_LDA,var_error_LDA,mean_error_decision_tree,var_error_decision_tree

In [ ]:
def exerciceI():
  #generation de données
  n=100
  rng = np.random.RandomState(2)
  dim =2
  X,y = sk.datasets.make_classification(n_samples=n, n_features=dim, n_redundant=0, n_informative=dim, n_repeated=0, n_clusters_per_class=1, random_state=rng)
  statistic_data(X,y)
  X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=.2, random_state=42)
  Xb=np.column_stack((X_train,np.ones(X_train.shape[0])))
  Xtb=np.column_stack((X_test,np.ones(X_test.shape[0])))

  figure = plt.figure()
  cm_bright = ListedColormap(['#FF0000', '#0000FF'])
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
  plt.title("Representation de données")

  #I.2: Calculer β , les paramètres de la frontière de décision
  beta,losses,reason=gradient_ascent(Xb,y_train)
  figure = plt.figure()
  plt.plot(losses) # affichage de la perte (au fur et a mesure des des itérations)
  plt.title("Pertes au fil du temp")
  print(reason)
  print("beta=",beta)

  #I.6: Comparer avec LDA
  w,b=LDA(X_train,y_train)
  print("w=",w,", b=",b)
  print("Error ratio of model generated by LDA on training data: ",calculate_LDA_error_ratio(w,b,X_train,y_train))
  print("Error ratio of model generated by LDA on testing data: ",calculate_LDA_error_ratio(w,b,X_test,y_test))

      #I.5: Taux d'erreur du modèle Logistic Regression
  print("Error ratio of model generated by Logistic Regression on training data: ",calculate_logistic_regression_error_ratio(beta,Xb,y_train))
  print("Error ratio of model generated by Logistic Regression on testing data: ",calculate_logistic_regression_error_ratio(beta,Xtb,y_test))

  clf = DecisionTreeClassifier(random_state=0)
  clf.fit(X_train, y_train)
  print("Error ratio of model generated by Decision Tree on training data: ",calculate_decision_tree_error_ratio(clf,X_train,y_train))
  print("Error ratio of model generated by Decision Tree on testing data: ",calculate_decision_tree_error_ratio(clf,X_test,y_test))

      #I.3: prédire la classe du point x [0,0]
  x=[0,0]
  xb=np.append(x,1)
  predict_class_of_point_using_logistic_regression(beta,xb)
  predict_class_of_point_using_LDA(w,b,x)

  draw_decision_boundary_LDA(w,b,X,y,"X1","X2")
        #I.4: frontière de décision de régression logistique
  draw_decision_boundary_logistic_regression(beta,X,y,"X1","X2")

  mean_error_logistic_regression,var_error_logistic_regression,mean_error_LDA,var_error_LDA,mean_error_decision_tree,var_error_decision_tree=report_error_ratio(X,y)
  print("Mean errors Logistic Regression", mean_error_logistic_regression)
  print("Var errors Logistic Regression", var_error_logistic_regression)
  print("Mean errors LDA", mean_error_LDA)
  print("Var errors LDA", var_error_LDA)
  print("Mean errors Decision Tree", mean_error_decision_tree)
  print("Var errors Decision Tree", var_error_decision_tree)

exerciceI()

Number of observations:  100
Number of features:  2
Is distribution of X approximately normal in class 0:  [ True  True]
Is distribution of X approximately normal in class 1:  [ True False]
attain itermax
beta= [[7.09905713]
 [2.83597819]
 [0.19270779]]
w= [-7.30352402 -2.66269449] , b= 2.842390511575443
Error ratio of model generated by LDA on training data:  0.0375
Error ratio of model generated by LDA on testing data:  0.05
Error ratio of model generated by Logistic Regression on training data:  0.025
Error ratio of model generated by Logistic Regression on testing data:  0.05
Error ratio of model generated by Decision Tree on training data:  0.0
Error ratio of model generated by Decision Tree on testing data:  0.15
Logistic regression says that x belongs to class 1
LDA says that x belongs to class 0


In [ ]:
def exerciceII():
  #generation de données
  n=100
  rng = np.random.RandomState(2)
  dim =2
  X,y = sk.datasets.make_classification(n_samples=n, n_features=dim, n_redundant=0, n_informative=dim, n_repeated=0, n_clusters_per_class=1, random_state=rng)
  statistic_data(X,y)
  X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=.2, random_state=42)
  Xb=np.column_stack((X_train,np.ones(X_train.shape[0])))
  Xtb=np.column_stack((X_test,np.ones(X_test.shape[0])))

  # inclure cette ligne pour la question II.7
  # II.7: Comparez les résultats de vos codes de régression logistique et de LDA sur ces données
  X_new_train=copy.deepcopy(X_train)
  X_new_train[0,:] = [5,5]
  Xb_new=np.column_stack((X_new_train,np.ones(X_new_train.shape[0])))

  beta,losses,reason=gradient_ascent(Xb_new,y_train)
  w,b=LDA(X_new_train,y_train)
  print("new beta=",beta)
  print(reason)
  plt.plot(losses) # affichage de la perte (au fur et a mesure des des itérations)

  print("new w=",w,", new b=",b)
  print("Error ratio of new model generated by LDA on new training data: ",calculate_LDA_error_ratio(w,b,X_new_train,y_train))
  print("Error ratio of new model generated by LDA on testing data: ",calculate_LDA_error_ratio(w,b,X_test,y_test))

  print("Error ratio of new model generated by Logistic Regression on new training data: ",calculate_logistic_regression_error_ratio(beta,Xb_new,y_train))
  print("Error ratio of new model generated by Logistic Regression on new testing data: ",calculate_logistic_regression_error_ratio(beta,Xtb,y_test))

  clf = DecisionTreeClassifier(random_state=0)
  clf.fit(X_train, y_train)
  print("Error ratio of model generated by Decision Tree on new training data: ",calculate_decision_tree_error_ratio(clf,X_train,y_train))
  print("Error ratio of model generated by Decision Tree on new testing data: ",calculate_decision_tree_error_ratio(clf,X_test,y_test))

  x=[0,0]
  xb=np.append(x,1)
  predict_class_of_point_using_logistic_regression(beta,xb)
  predict_class_of_point_using_LDA(w,b,x)

  new_X= np.concatenate((X_new_train, X_test), axis=0)
  new_y= np.concatenate((y_train, y_test), axis=0)
  draw_decision_boundary_LDA(w,b,new_X,new_y,"X1","X2")
  draw_decision_boundary_logistic_regression(beta,new_X,new_y,"X1","X2")

  #II.8: Stratégie de validation croisée à n blocs et reporter les taux d'erreur (moyenne et variance) des deux modèles.
  mean_error_logistic_regression,var_error_logistic_regression,mean_error_LDA,var_error_LDA,mean_error_decision_tree,var_error_decision_tree=report_error_ratio(X,y)
  print("Mean errors Logistic Regression", mean_error_logistic_regression)
  print("Var errors Logistic Regression", var_error_logistic_regression)
  print("Mean errors LDA", mean_error_LDA)
  print("Var errors LDA", var_error_LDA)
  print("Mean errors Decision Tree", mean_error_decision_tree)
  print("Var errors Decision Tree", var_error_decision_tree)

exerciceII()

In [ ]:
def exerciceII9():
  #II.9: Modifier le code de génération de données (en augmentant le nombre de dimensions et en ajoutant des variables correlées). Comparer le comportement des deux modèles
  n=100
  rng = np.random.RandomState(2)
  dim=7
  X, y = sk.datasets.make_classification(
      n_samples=n,
      n_features=dim,
      n_redundant=2,
      n_informative=2,
      n_clusters_per_class=1,
      random_state=rng
  )
  statistic_data(X,y)

  #pour pouvoir représenter les données sous forme de 2D, j'ai réduit le nombre d'attributs en utilisant PCA
  #(Suite à ce que je connais, on pourra utiliser w généné par LDA normalement pour faire ça aussi, mais faute de temps, je n'ai pas pu faire des recherches en détail sur ça.
  #Donc, j'ai utilisé une fonction qui déjà existe dans la librairie sklearn pour la sûreté)
  pca = PCA(n_components=2)
  transformed_X = pca.fit_transform(X,y)
  transformed_Xb = np.column_stack((transformed_X,np.ones(transformed_X.shape[0])))

  X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=.2, random_state=42)

  Xb=np.column_stack((X_train,np.ones(X_train.shape[0])))
  Xtb=np.column_stack((X_test,np.ones(X_test.shape[0])))

  figure = plt.figure()
  cm_bright = ListedColormap(['#FF0000', '#0000FF'])
  plt.scatter(transformed_X[:, 0], transformed_X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
  plt.show()

  beta,losses,reason=gradient_ascent(Xb,y_train)
  w,b=LDA(X_train,y_train)

  #Les paramètres de 2 modèles sur les données réduites par PCA, sertent à représenter les frontières de décision en 2D
  transformed_beta,_,_=gradient_ascent(transformed_Xb,y)
  transformed_w,transformed_b=LDA(transformed_X,y)

  print("beta=",beta)
  print(reason)
  figure = plt.figure()
  plt.plot(losses) # affichage de la perte (au fur et a mesure des des itérations)

  print("w=",w,", b=",b)
  print("Error ratio of new model generated by LDA on training data: ",calculate_LDA_error_ratio(w,b,X_train,y_train))
  print("Error ratio of new model generated by LDA on testing data: ",calculate_LDA_error_ratio(w,b,X_test,y_test))

  print("Error ratio of new model generated by Logistic Regression on training data: ",calculate_logistic_regression_error_ratio(beta,Xb,y_train))
  print("Error ratio of new model generated by Logistic Regression on testing data: ",calculate_logistic_regression_error_ratio(beta,Xtb,y_test))

  clf = DecisionTreeClassifier(random_state=0)
  clf.fit(X_train, y_train)
  print("Error ratio of model generated by Decision Tree on training data: ",calculate_decision_tree_error_ratio(clf,X_train,y_train))
  print("Error ratio of model generated by Decision Tree on testing data: ",calculate_decision_tree_error_ratio(clf,X_test,y_test))

  draw_decision_boundary_LDA(transformed_w,transformed_b,transformed_X,y,"PC1","PC2")
  draw_decision_boundary_logistic_regression(transformed_beta,transformed_X,y,"PC1","PC2")

  stat_0, p_value_0 = stats.normaltest(X[y==0])
  stat_1, p_value_1 = stats.normaltest(X[y==1])
  print(p_value_0>= 0.05)
  print(p_value_1>= 0.05)

  mean_error_logistic_regression,var_error_logistic_regression,mean_error_LDA,var_error_LDA,mean_error_decision_tree,var_error_decision_tree=report_error_ratio(X,y)
  print("Mean errors Logistic Regression", mean_error_logistic_regression)
  print("Var errors Logistic Regression", var_error_logistic_regression)
  print("Mean errors LDA", mean_error_LDA)
  print("Var errors LDA", var_error_LDA)
  print("Mean errors Decision Tree", mean_error_decision_tree)
  print("Var errors Decision Tree", var_error_decision_tree)

exerciceII9()

In [ ]:
def exerciceIII():
  scaler = StandardScaler()

  #génération et traitement de données. Choisir que les colonnes ayant values de type "number". Remplacer toutes les valeurs NaN avec la moyenne de leurs colonne.
  df = pd.read_csv('./heart.csv')
  df = df.select_dtypes(include=np.number)
  df = df.fillna(df.mean())
  dfy=df['target']
  dfx=df.drop("target", axis=1)
  X = scaler.fit_transform(dfx)
  y=dfy.to_numpy()
  statistic_data(X,y)

  pca = PCA(n_components=2)
  transformed_X = pca.fit_transform(X,y)
  transformed_Xb = np.column_stack((transformed_X,np.ones(transformed_X.shape[0])))

  X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=.2, random_state=42)

  Xb=np.column_stack((X_train,np.ones(X_train.shape[0])))
  Xtb=np.column_stack((X_test,np.ones(X_test.shape[0])))

  figure = plt.figure()
  cm_bright = ListedColormap(['#FF0000', '#0000FF'])
  plt.scatter(transformed_X[:, 0], transformed_X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
  plt.show()

  beta,losses,reason=gradient_ascent(Xb,y_train)
  w,b=LDA(X_train,y_train)
  clf = DecisionTreeClassifier(random_state=0)
  clf.fit(X_train, y_train)

  transformed_beta,_,_=gradient_ascent(transformed_Xb,y)
  transformed_w,transformed_b=LDA(transformed_X,y)
  transformed_clf = DecisionTreeClassifier(random_state=0)
  transformed_clf.fit(transformed_X, y)

  figure = plt.figure()
  plt.plot(losses) # affichage de la perte (au fur et a mesure des des itérations)
  print(reason)
  print("beta=",beta)

  print("w=",w,", b=",b)
  print("Error ratio of new model generated by LDA on training data: ",calculate_LDA_error_ratio(w,b,X_train,y_train))
  print("Error ratio of new model generated by LDA on testing data: ",calculate_LDA_error_ratio(w,b,X_test,y_test))

  print("Error ratio of new model generated by Logistic Regression on training data: ",calculate_logistic_regression_error_ratio(beta,Xb,y_train))
  print("Error ratio of new model generated by Logistic Regression on testing data: ",calculate_logistic_regression_error_ratio(beta,Xtb,y_test))

  print("Error ratio of new model generated by Decision Tree on training data: ",calculate_decision_tree_error_ratio(clf,X_train,y_train))
  print("Error ratio of new model generated by Decision Tree on testing data: ",calculate_decision_tree_error_ratio(clf,X_test,y_test))

  draw_decision_boundary_LDA(transformed_w,transformed_b,transformed_X,y,"PC1","PC2")
  draw_decision_boundary_logistic_regression(transformed_beta,transformed_X,y,"PC1","PC2")
  draw_decision_boundary_decision_tree(transformed_clf,transformed_X,y,"PC1","PC2")

  mean_error_logistic_regression,var_error_logistic_regression,mean_error_LDA,var_error_LDA,mean_error_decision_tree,var_error_decision_tree=report_error_ratio(X,y)

  print("Mean errors Logistic Regression", mean_error_logistic_regression)
  print("Var errors Logistic Regression", var_error_logistic_regression)
  print("Mean errors LDA", mean_error_LDA)
  print("Var errors LDA", var_error_LDA)
  print("Mean errors Decision Tree", mean_error_decision_tree)
  print("Var errors Decision Tree", var_error_decision_tree)

exerciceIII()